In [11]:
from fastapi import FastAPI, File, UploadFile
from fastapi.middleware.cors import CORSMiddleware
import requests
import shutil
import joblib
import pandas as pd
import numpy as np
import librosa
import soundfile

#Load an audio file and transform it
def x_pred_preprocessing(audio_path):
    x_pred_preprocessed = extract_features(audio_path,
                                           mfcc=True,
                                           chroma=False,
                                           mel=True,
                                           temp=True)
    x_pred_preprocessed = x_pred_preprocessed.reshape(1, 552)
    return x_pred_preprocessed


#Predict the emotion
def return_predict(x_pred_preprocessed, model_path='../MLP_model.joblib'):
    model = joblib.load(model_path)
    prediction = model.predict(x_pred_preprocessed)
    return prediction[0]


#Return a dataframe giving the predicted probabilities for each emotion in observed_emotions
def predict_proba(observed_emotions, x_pred_preprocessed, model_path='../MLP_model.joblib'):
    model = joblib.load(model_path)
    emotion_list = observed_emotions
    emotion_list.sort()
    model_pred_prob = pd.DataFrame((model.predict_proba(x_pred_preprocessed) * 100).round(2),
                                columns=emotion_list)
    return model_pred_prob


#Extract features (mfcc, chroma, mel, temp) from a sound file
def extract_features(file_name, mfcc, chroma, mel, temp):
    with soundfile.SoundFile(file_name) as sound_file:
        X = sound_file.read(dtype="float32")
        sample_rate = sound_file.samplerate
        if chroma:
            stft = np.abs(librosa.stft(X))
        result = np.array([])
        if mfcc:
            mfccs = np.mean(librosa.feature.mfcc(y=X,
                                                 sr=sample_rate,
                                                 n_mfcc=40).T,
                            axis=0)
            result = np.hstack((result, mfccs))
        if chroma:
            chroma = np.mean(librosa.feature.chroma_stft(S=stft,
                                                         sr=sample_rate).T,
                             axis=0)
            result = np.hstack((result, chroma))
        if mel:
            mel = np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,
                          axis=0)
            result = np.hstack((result, mel))
        if temp:
            temp = np.mean(librosa.feature.tempogram(y=X, sr=sample_rate).T,
                           axis=0)
            result = np.hstack((result, temp))
    return result


In [12]:
filename='/Users/pankajpatel/code/pankaj-lewagon/ser/03-01-03-01-02-01-23.wav'

In [68]:
observed_emotions = ['calm', 'happy', 'sad', 'angry', 'fearful', 'disgust']

x_pred_preprocessed = x_pred_preprocessing(filename)
prediction = return_predict(x_pred_preprocessed)
pedicted_probas =predict_proba(observed_emotions, x_pred_preprocessed)

/Users/pankajpatel/.pyenv/versions/3.8.6/envs/ser_proj/lib/python3.8/site-packages/librosa/filters.py:238: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  warnings.warn(


In [66]:
prediction

'happy'

In [70]:
pedicted_probas.T


,0
angry,0.10
calm,0.06
disgust,0.01
fearful,2.19
happy,96.58
sad,1.06


In [56]:
predicted_probas =predict_proba(observed_emotions, x_pred_preprocessed).T
predicted_probas.rename(columns={0 : "probability"}, inplace=True)
probas=pd.DataFrame.to_dict(predicted_probas)
probas['emotion']={0:prediction}

In [57]:
probas

{'probability': {'angry': 0.1,
  'calm': 0.06,
  'disgust': 0.01,
  'fearful': 2.19,
  'happy': 96.58,
  'sad': 1.06},
 'emotion': {0: 'happy'}}

In [28]:
predicted_probas =predict_proba(observed_emotions, x_pred_preprocessed).T


In [41]:
predicted_probas.rename(columns={0 : "probability"}, inplace=True)

predicted_probas

,probability
angry,0.10
calm,0.06
disgust,0.01
fearful,2.19
happy,96.58
sad,1.06


In [48]:
probas=pd.DataFrame.to_dict(predicted_probas)

In [49]:
probas

{'probability': {'angry': 0.1,
  'calm': 0.06,
  'disgust': 0.01,
  'fearful': 2.19,
  'happy': 96.58,
  'sad': 1.06}}

In [52]:

xx['emotion']

In [53]:
probas

{'probability': {'angry': 0.1,
  'calm': 0.06,
  'disgust': 0.01,
  'fearful': 2.19,
  'happy': 96.58,
  'sad': 1.06},
 'emotion': ['happy']}